1. get sample image
2. download image
3. perform imnage grouping
4. inspect result

Next items to do, prepare and package as jobs
1. create auto-query
2. create sql to store result
3. export to executable py


# Plot sse against k
plt.figure(figsize=(6, 6))
plt.plot(list_k, sse)
plt.xlabel(r'Number of clusters *k*')
plt.ylabel('Sum of squared distance');

# prepare data for saving to rdbms
# print(clus_new)
# i = 0 
result_cluster = []

for item in clus_new:
    # print(clus_new[item])
    print(len(clus_new[item]))
    for file in clus_new[item]:
        # print(file, item)
        result_cluster.append(file)
    # i = i + 1

# view_cluster_2(0)
len(clus_new)
len(result_cluster)


 * Kesepakatan status di kolom screen_analisis_ai.status
 * 1 --> baru diinput
 * 2 --> lagi dikerjakan
 * 3 --> proses berhasil
 * 4 --> proses gagal
 *
 
 * Kesepakatan jenis analisa AI
 * 1 --> Analisa Cluster
 * 2 --> Analisa image clustering
 * 3 --> Analisa sentiment


In [1]:
# for loading/processing the images  
from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array 
from keras.applications.vgg16 import preprocess_input 
from sklearn.metrics.pairwise import cosine_similarity

# models 
from keras.applications.vgg16 import VGG16 
from keras.models import Model

# clustering and dimension reduction
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# for everything else
import os
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import pickle
import requests
from pathlib import Path
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
from PIL import Image
from io import BytesIO
from urllib.parse import urlparse

2024-04-09 17:08:04.412839: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-09 17:08:04.415323: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-09 17:08:04.464179: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-09 17:08:04.465080: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2024-04-09 17:08:05.354376: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def resize_and_upload_image(url, save_folder):
    try:
        # Fetch the image from the URL
        response = requests.get(url)
        response.raise_for_status()

        # Open the image from the response content
        image = Image.open(BytesIO(response.content))

        # Check the image size in bytes
        image_size = len(response.content)

        # Set the maximum allowed size in bytes (200KB)
        max_size = 200 * 1024

        # Extract the base path from the URL for saving the images
        parsed_url = urlparse(url)
        base_path = os.path.basename(parsed_url.path)

        # Determine the paths for saving the images
        original_image_path = os.path.join(save_folder, 'original_' + base_path)
        resized_image_path = os.path.join(save_folder, 'resized_' + base_path)

        # If the image size exceeds the maximum allowed size, resize it
        if image_size > max_size:
            print('more than 200kb')
            print(original_image_path)
            print(resized_image_path)
            
            # Calculate the scaling factor to reduce the image size
            scale_factor = (max_size / image_size) ** 0.5
            new_width = int(image.width * scale_factor)
            new_height = int(image.height * scale_factor)

            # Resize the image
            resized_image = image.resize((new_width, new_height), Image.ANTIALIAS)

            # Save the resized image to the local folder
            resized_image.save(resized_image_path, format="JPEG")

            # Reupload the resized image to the original location
            with open(resized_image_path, 'rb') as resized_file:
                reupload_url = url  # Use the original URL for reupload
                print(url)
                files = {'file': ('resized_' + base_path, resized_file)}
                reupload_response = requests.post(reupload_url, files=files)
                print(reupload_response)

            # Delete the original image in the local folder
            os.remove(original_image_path)

        else:
            # Image is smaller than 200KB, no need to resize or reupload
            resized_image_path = original_image_path

        return original_image_path, resized_image_path

    except Exception as e:
        # Handle errors such as invalid URLs or image processing issues
        return str(e)

# Example usage:
# imageURL = "https://example.com/path/to/your/image.jpg"
# save_folder = "path/to/your/local/folder"
# original_image_path, resized_image_path = resize_and_upload_image(imageURL, save_folder)
# Both original_image_path and resized_image_path will contain the file paths in the local folder.

def download_image(url, folder_path):
    try:
        # Send an HTTP GET request to the URL
        response = requests.get(url)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Extract the filename from the URL
            filename = os.path.join(folder_path, os.path.basename(url))

            # Save the image to the specified folder location
            with open(filename, 'wb') as file:
                file.write(response.content)
            file_size_bytes = os.path.getsize(filename)
            file_size_kb = file_size_bytes / 1024

            if file_size_kb >= 200:
                print("Resizing...")
                resize_and_upload_image(filename,url)
            # print(f"Image downloaded and saved as {filename}")
        else:
            print(f"Failed to download image. Status code: {response.status_code}")

    except Exception as e:
        print(f"Error: {str(e)}")

def resize_and_upload_image(file_path, upload_url, target_size_kb=200):
    try:
        # Open the image from the file
        with Image.open(file_path) as image:
            # Calculate the target size in bytes
            target_size_bytes = target_size_kb * 1024

            # Initialize the quality variable
            quality = 95

            while os.path.getsize(file_path) > target_size_bytes:
                # Resize the image while keeping the quality constant
                width, height = image.size
                new_width = int(width * 0.9)
                new_height = int(height * 0.9)

                # Ensure the dimensions are at least 1
                new_width = max(1, new_width)
                new_height = max(1, new_height)

                image = image.resize((new_width, new_height), Image.LANCZOS)

                # Save the resized image to the same file
                image.save(file_path, "JPEG", quality=quality)

            if os.path.getsize(file_path) <= target_size_bytes:
                print(f"Image resized and overwritten at: {file_path}")

                # Upload the resized image to the given URL
                with open(file_path, 'rb') as file:
                    response = requests.post(upload_url, files={'file': (os.path.basename(file_path), file)})

                if response.status_code == 200:
                    print(f"Image uploaded to {upload_url}")
                else:
                    print(f"Upload failed with status code {response.status_code}")
            else:
                print(f"Image not resized, not uploaded.")
                
    except Exception as e:
        print(f"Error: {e}")

## query functions


import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, MetaData, Table
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy.util import deprecations
from sqlalchemy.sql import text
deprecations.SILENCE_UBER_WARNING = True

def execute_query_psql(query, params=None):
    # Set your PostgreSQL connection parameters
    db_params = {
        'host': '98.98.117.105',
        'port': '5432',
        'database': 'medols',
        'user': 'postgres',
        'password': 'FEWcTB3JIX5gK4T06c1MdkM9N2S8w9pb',
    }

    # Create a SQLAlchemy engine
    engine = create_engine(f"postgresql+psycopg2://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['database']}")

    # Create a metadata object
    metadata = MetaData()

    # Create a session
    Session = sessionmaker(bind=engine)
    session = Session()

    try:
        # Execute the query with optional parameters
        result = session.execute(text(query), params)

        # Check if the query is a SELECT query
        is_select_query = result.returns_rows

        if is_select_query:
            # Fetch the data and return as a Pandas DataFrame
            columns = result.keys()
            fetched_data = result.fetchall()
            df = pd.DataFrame(fetched_data, columns=columns)
            # print("Fetched Data as DataFrame:")
            # print(df)
            return df
        else:
            # Get the number of rows affected for non-SELECT queries
            rows_affected = result.rowcount

            # Commit the changes to the database for non-SELECT queries
            session.commit()

            print(f"Query executed successfully. {rows_affected} rows affected.")
            return rows_affected
    except Exception as e:
        # Rollback changes if there's an error
        session.rollback()
        print(f"Error executing query: {e}")
    finally:
        # Close the session
        session.close()

def clear_image():
    folder_path = '/home/jup_user/multipool/ai-case-study/img'
    # Create a Path object for the folder
    folder = Path(folder_path)
    
    # Iterate through the files in the folder and delete them
    for file in folder.iterdir():
        try:
            if file.is_file():
                file.unlink()
                # print(f"Deleted {file}")
            else:
                print(f"{file} is not a file.")
        except Exception as e:
            print(f"Error deleting {file}: {str(e)}")

# extracting feature from image files
def extract_features(file, model):
    # load the image as a 224x224 array
    img = load_img(file, target_size=(224,224))
    # convert from 'PIL.Image.Image' to numpy array
    img = np.array(img) 
    # reshape the data for the model reshape(num_of_samples, dim 1, dim 2, channels)
    reshaped_img = img.reshape(1,224,224,3) 
    # prepare image for model
    imgx = preprocess_input(reshaped_img)
    # get the feature vector
    features = model.predict(imgx, use_multiprocessing=True)
    return features

# function that lets you view a cluster (based on identifier)        
def view_cluster(cluster):
    plt.figure(figsize = (25,25));
    # gets the list of filenames for a cluster
    files = groups[cluster]
    # only allow up to 30 images to be shown at a time
    if len(files) > 30:
        print(f"Clipping cluster size from {len(files)} to 30")
        files = files[:29]
    # plot each image in the cluster
    for index, file in enumerate(files):
        plt.subplot(10,10,index+1);
        img = load_img(file)
        img = np.array(img)
        plt.imshow(img)
        plt.axis('off')

In [3]:
#
# Get jobsid from queue table
# 
iJobID = 0
sql = """
select 	a.id, a.hastag, a.parameter
from 	screen_analisis_ai a inner join ret_available_db b 
		on a.hastag = b.db_id 
where 	a.active = 1
		and a.status = 1
		and a.jenis_analisa = '2'
order by b.since desc limit 1
"""

df_res = execute_query_psql(sql)

# check job availability
if(len(df_res)) == 0:
    # get out, nothing to do
    print('Zero jobs, quitting now')
    quit()

df_res.head()
iJobID = df_res['hastag'][0]
const_job_id = df_res['id'][0]
const_parameter = df_res['parameter'][0]

print('iJobID: ' + str(iJobID))
print('const_job_id: ' + str(const_job_id))
print('const_parameter: ' + str(const_parameter))


iJobID: 87596
const_job_id: 3846
const_parameter: None


In [4]:
## report to dbms that we are working on this row
sql = ""
sql = "update screen_analisis_ai set status = 2, last_status_update = now(), start_process = now() where id = '" + str(const_job_id) + "';"
print(sql)
# execute
# execute_sqlalchemy_transaction(sql)
execute_query_psql(sql)
#
# Create Header Record
sql = "insert into ret_analysis_header (job_id, datetime_start, user_id) values (%s, now(), %s)" % (const_job_id,"1")
# Execute the query
execute_query_psql(sql)


update screen_analisis_ai set status = 2, last_status_update = now(), start_process = now() where id = '3846';
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


1

In [5]:
## get images from server
# Example usage:
query = """
select a.id, a.db_id, c.tweet_id, c.filename
from ret_available_db a inner join ret_tweet b on a.db_id = b.db_id
inner join media_files c on b.id = c.tweet_id
where 	a.db_id = %s
"""
query = query % str(iJobID)
print(query)

result_df = execute_query_psql(query)

## downloading image set
img_prefix_http = "http://98.98.117.121:8082/api/media/photo/download/twitter/"
folder_path = "/home/jup_user/multipool/ai-case-study/img"    # Replace with the desired folder path

# download_image(url, folder_path)
# clear up folder first
clear_image()

i = 0
for index, row in result_df.iterrows():
    # print(result_df.at[index,'filename'])
    url = img_prefix_http + result_df.at[index,'filename']
    print(url)
    download_image(url, folder_path)
    i = i +1

print("finished downloading " + str(i) + " image set" )


select a.id, a.db_id, c.tweet_id, c.filename
from ret_available_db a inner join ret_tweet b on a.db_id = b.db_id
inner join media_files c on b.id = c.tweet_id
where 	a.db_id = 87596



http://98.98.117.121:8082/api/media/photo/download/twitter/GKoOcKmacAAMEkX.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKoOO52bAAA0yQc.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKoNqWgacAAgobq.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKoNoNDasAAQgiw.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKoM1z0bsAEEd-e.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKoNRdLbUAEy1Wr.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKoM6kvbcAEUCSd.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKoMmkZbEAA3e8N.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKoLVBPakAA2Def.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKiDzdibcAAvD6e.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKoKenZbQAAoxRQ.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKoKNtNbkAAdTYa.jpg
http://98.98.117.121:8082/api/media/photo/download/t

http://98.98.117.121:8082/api/media/photo/download/twitter/GKn6VMSawAAG8kV.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/E8WUcgXVoAQosQN.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKm59u4awAAFu_Y.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKgSMXiaIAAInbe.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKn3DTMbAAAb6ns.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKkrJnJaMAA-FP9.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKn24GdacAAfzli.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKnzOYHbUAAb9I4.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKnzSpAbYAAMqFJ.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKny0yuaMAAi98w.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKny7J6agAAtDdi.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKjEHbJbgAANLjg.jpg
http://98.98.117.121:8082/api/media/photo/download/t

http://98.98.117.121:8082/api/media/photo/download/twitter/GKnBntUbwAAI9oV.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKnA6EQaEAAbxzU.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKnAMP5bgAAO-D3.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKk1hzlbIAATYhY.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKm-hmXaAAAOM-F.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKm8yNCaUAArETs.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKm8Xi7bQAAM3Zo.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKm7m4xbgAAzTFR.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKm7awtbIAAL49O.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKiJuFIa4AIppc-.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKkEElGbIAAvVhc.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKm2VqgaoAAQSmx.jpg
http://98.98.117.121:8082/api/media/photo/download/t

http://98.98.117.121:8082/api/media/photo/download/twitter/GKlwhH8bAAAf9z4.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKlv4ZVbIAAl_i9.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKlvhnkaoAArVQk.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKlvFqObsAAz2oX.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKj2K1-aoAA8vBX.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKj1yCCaEAMmE7X.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKkbOgUbAAAR-1a.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKkaZz-aQAAtzxP.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKlm5Sma0AA1QI1.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKlmkc7bgAA3j-j.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKiGlpJbAAAiGfX.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKljO9uaUAAoWpU.jpg
http://98.98.117.121:8082/api/media/photo/download/t

http://98.98.117.121:8082/api/media/photo/download/twitter/GKlBI87aUAA9uKj.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKlAqwaagAIPLFH.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKk_0JGboAAgrzT.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKk_yzrbwAAhh1o.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKk_oELbYAADV_c.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKk_vb5bUAAuIAW.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKk9173asAIz6qB.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKk9o44b0AAc86G.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKk8KsvbsAAztMa.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKk8uuYaIAAkAWG.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKk8onzbkAEWmqG.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKk7NMSakAA1nZL.jpg
http://98.98.117.121:8082/api/media/photo/download/t

http://98.98.117.121:8082/api/media/photo/download/twitter/GKkoqWzbEAAbzOC.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKkokcDaQAAxmr0.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKkokXZawAAVYNx.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKkoaXXaoAASR9Y.jpg
http://98.98.117.121:8082/api/media/photo/download/twitter/GKkoRDfaMAANEBZ.jpg
finished downloading 281 image set


In [6]:
path = folder_path
# change the working directory to the path where the images are located
os.chdir(path)

# this list holds all the image filename
flowers = []

# creates a ScandirIterator aliased as files
with os.scandir(path) as files:
  # loops through each file in the directory
    for file in files:
        if file.name.endswith('.jpg'):
          # adds only the image files to the flowers list
            flowers.append(file.name)
        if file.name.endswith('.png'):
            # adds only the image files to the flowers list
            flowers.append(file.name)
            
model = VGG16()
model = Model(inputs = model.inputs, outputs = model.layers[-2].output)
   
data = {}
p = "/home/jup_user/multipool/ai-case-study/img"

# lop through each image in the dataset
for flower in flowers:
    # try to extract the features and update the dictionary
    try:
        feat = extract_features(flower,model)
        data[flower] = feat
    # if something fails, save the extracted features as a pickle file (optional)
    except:
        with open(p,'wb') as file:
            pickle.dump(data,file)
          
# get a list of the filenames
filenames = np.array(list(data.keys()))

# get a list of just the features
feat = np.array(list(data.values()))

# reshape so that there are 210 samples of 4096 vectors
feat = feat.reshape(-1,4096)

# get the unique labels (from the flower_labels.csv)
df = pd.read_csv('/home/jup_user/multipool/ai-case-study/flower_labels.csv')
label = df['label'].tolist()
unique_labels = list(set(label))

# reduce the amount of dimensions in the feature vector
pca = PCA(n_components=100, random_state=22)
pca.fit(feat)
x = pca.transform(feat)

# cluster feature vectors
kmeans = KMeans(n_clusters=len(unique_labels), random_state=22)
kmeans.fit(x)

# holds the cluster id and the images { id: [images] }
groups = {}
for file, cluster in zip(filenames,kmeans.labels_):
    if cluster not in groups.keys():
        groups[cluster] = []
        groups[cluster].append(file)
    else:
        groups[cluster].append(file)

# this is just incase you want to see which value for k might be the best 
sse = []
list_k = list(range(3, 50))

for k in list_k:
    km = KMeans(n_clusters=k, random_state=22)
    km.fit(x)
    
    sse.append(km.inertia_)

# function to calculate clusters
def cluster(filePaths, features, threshold=0.9):
    features = features.reshape(-1,4096)
    simMatrix = cosine_similarity(features)
    clusters = {}
    for i in range(len(features)):
        dupIdx = list(np.where(simMatrix[i] > threshold)[0])
        # The similarity matrix will include comparisons of items with themselves, which will always 
        # result in a similarity of 1.0 (100%) and is redundant, so we ignore those
        if len(dupIdx) > 1:
            curCluster, clusterMatch = list(dupIdx), None
            # The first time an image is found to be in any given cluster, we log the entire cluster, 
            # so subsequent checks of other images from the same cluster would result in duplicated clusters.
            # Check for that here
            for cIdx in clusters:
                if curCluster[0] in clusters[cIdx]:
                    clusterMatch = cIdx
                    break
            # If the current cluster didn't match any existing ones, create/log it
            if clusterMatch == None: clusters[len(clusters)] = curCluster
    # Resolve file indices back to file paths
    for cIdx in clusters: clusters[cIdx] = [filePaths[x] for x in clusters[cIdx]]
    return clusters

# another method of clustering based on CSI
clus_new = cluster(filenames, feat, threshold=0.6)

1/1 [==============================] - ETA: 0s

1/1 [==============================] - 0s 248ms/step


1/1 [==============================] - ETA: 0s

1/1 [==============================] - 0s 74ms/step


1/1 [==============================] - ETA: 0s

...

/home/jup_user/mlproject/jup-env/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


/home/jup_user/mlproject/jup-env/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


/home/jup_user/mlproject/jup-env/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


/home/jup_user/mlproject/jup-env/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


/home/jup_user/mlproject/jup-env/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [7]:
# prepare data for saving to rdbms
save_df = result_df

In [8]:
# save_df[["file_folder","filename_actual"]] = save_df['filename'].str.split('/',n=1, expand=True)
save_df['cluster_number'] = 0

In [9]:
for item in clus_new:
    # print(clus_new[item])
    # print(len(clus_new[item]))
    for file in clus_new[item]:
        # print(item)
        print(file)
        filter_condition = save_df['filename'] == file
        save_df.loc[filter_condition,'cluster_number'] = item
        result = save_df[filter_condition]
        # result['cluster_number'] = item
        # print(save_df.loc[filter_condition,'cluster_number'])

GKlH0RwbIAAQcSR.jpg
GKlF0qxbsAAgNoe.jpg
GKkoRDfaMAANEBZ.jpg
GKl8DxtbMAAzZ79.jpg
GKkrJnJaMAA-FP9.jpg
GKl4T_rboAAwhIu.jpg
GKiDzdibcAAvD6e.jpg
GKmemYja4AA-iS9.jpg
GKk_oELbYAADV_c.jpg
GKktScDagAAfqeN.png
GKjUuOEbwAAa037.jpg
GKmhzXNacAAH2Fx.png
GKkmrd0asAAtOU7.jpg
GKmXQR-aoAAaCzp.jpg
GKlRf7_aoAAjYlE.jpg
GKnILyda0AAIv7B.jpg
GKmXUgzaIAAM14L.jpg
GKkNwLbaUAAS1mE.jpg
GKmX5IxbAAAQ4SO.jpg
GKk3PJMawAA0wh3.jpg
GKk9o44b0AAc86G.jpg
GKmloa1aQAAJPGS.jpg
GKlh0EzbEAA7crZ.jpg
GKlKunQbwAAs0gd.jpg
GKoJPlIawAEYaUE.jpg
GKl70XIbgAAMSoW.jpg
GKnI55oboAAnhCp.jpg
GKlKskFaQAApWvs.jpg
GKnAMP5bgAAO-D3.jpg
GKk1hzlbIAATYhY.jpg
GKk9173asAIz6qB.jpg
GKm59u4awAAFu_Y.jpg
GKkxYWPaIAAOY1X.jpg
GKmhxvTbEAI1N3l.jpg
GKnlYYKacAAcc13.jpg
GKmNojrbAAAl4Zh.jpg
GKkqoNHbAAEDjPA.jpg
GKoHU3pbsAAVqjm.jpg
GKjbem5aQAA_JBb.jpg
GKnJ0fOaQAEv8Ts.jpg
GKlQMsSbIAAzhg3.jpg
GKk0acSbYAAo_y7.jpg
GKk7CMJbUAAEzFY.jpg
GKl71K0boAAoTT0.jpg
GKn24GdacAAfzli.jpg
GKkEzubasAAKqLA.jpg
GKlUiWaboAAJlEW.jpg
GKkoqWzbEAAbzOC.jpg
GKnGMtpaAAAejL3.jpg
GKlmkc7bgAA3j-j.jpg


In [10]:
## saving result to table
# jobid, tweet_id, cluster_no

s_cluster_number = ""

for index, row in save_df.iterrows():
    if row['cluster_number'] == '':
        s_cluster_number = "NULL"
    else:
        s_cluster_number = row['cluster_number']
        
    sql = "INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (%s, %s, %s)" % (str(const_job_id), row['tweet_id'], s_cluster_number)
    print(sql)
    execute_query_psql(sql)
    
    

INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1777248887318991223, 0)
Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1776897068943962120, 27)
Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1777248018942869673, 19)
Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1777248018942869673, 0)


Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1777247726398517733, 16)
Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1777247599910912040, 31)
Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1777247205642174661, 0)
Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1777246862346784889, 0)
Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1777245461587644919, 1)
Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1777245120741740966, 1)
Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 

Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1777243868557386049, 31)
Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1777243826547261910, 6)
Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1777243599333400818, 19)
Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1777243172105842899, 6)


Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1777243058683400464, 26)
Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1777241825604166039, 0)
Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1777241458820694437, 26)
Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1777241129404321980, 0)
Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1777241071946170444, 0)
Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1777241061158535575, 1)
Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 

Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1777239853098504510, 0)
Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1777238135455584628, 9)
Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1777236840325828906, 0)
Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1777236966536318996, 0)
Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1777235989770977687, 26)


...

Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1776984380357128674, 0)
Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1777007565060395309, 0)
Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1776813549920362899, 11)
Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1777006707513933983, 0)
Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1776813549920362899, 1)
Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1777004995726184785, 15)
Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 

Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1776983048002973836, 0)
Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1776998739753959791, 0)
Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1776998699622858965, 0)
Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1776813549920362899, 7)


Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1776998200190353869, 0)
Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1776998127188554105, 0)
Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1776998071718859087, 28)
Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1776997632147431882, 0)
Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1776997567731290145, 34)
Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1776997550085885986, 34)
Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846,

Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1776996329887289847, 35)
Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1776531563460891098, 7)
Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1776996174865879302, 0)


Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1776996139495256318, 0)
Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1776995967876894852, 0)
Query executed successfully. 1 rows affected.
INSERT into ret_cluster_result (job_id, tweet_id, cluster_no) values (3846, 1776995806979244509, 35)
Query executed successfully. 1 rows affected.


In [11]:
# closing .... report back job status into rdbms
sql = "update screen_analisis_ai set end_process = now(), status = 3, processby_id = 1 where id = %s" % (str(const_job_id))
execute_query_psql(sql)

sql = "update screen_analisis_ai set duration = EXTRACT(EPOCH FROM (end_process - start_process)) where id = " + str(const_job_id)
execute_query_psql(sql)

Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


1

In [12]:
# wait 10 seconds before finished
import time
time.sleep(10)